In [1]:
import openai
import re
import time
import json

import numpy as np
import os
from tqdm import tqdm
from pprint import pprint
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed

In [3]:
@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)

In [ ]:
social_test = []
datas = json.loads(open("./knowledge.json", "r").read())
for data in datas:
    if data['task'] != 'siqa':
        continue
    social_test.append(data)
print("len csqa-dev:", len(social_test))
print(social_test[0].keys())
print(social_test[0]["query"])

In [ ]:
prompt_original_direct = open('prompt_original_direct.txt').read()
print(prompt_original_direct)

In [ ]:
import time
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["A", "B", "C"]
for mention_idx, mention in tqdm(enumerate(social_test), total=len(social_test)):
    # answer_gold = mention["answerKey"].lower()
    answer = mention["answer"]
    prompt_q  = prompt_original_direct + "\nQ: " + mention["query"].split('\\n')[0]+ "\n"
    # prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(mention["cands"]):
        if answer == c:
            answer_gold = answer_id[idx]
        prompt_q += '(%c) %s\n' % (answer_id[idx], c)
    prompt_q += mention["query"].split('\\n')[-1].strip()
    prompt_q += "\nThe answer is "
    # print(prompt_q)
#     break
    while True:
        try:
            response = completion_with_backoff(model="davinci", 
                                                prompt=prompt_q, 
                                                temperature=0, 
                                                max_tokens=5,
                                            )
            break
        except:
            time.sleep(3)
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].strip().split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
#         print("gold:", answer_gold)
#         print("pred:", answer_pred)
#         print("acc:", acc)
#         print("total:", total)
#         print(prompt_q)
#         print(response)
#         break

print('Total %d correct %d acc %.4f' % (total , acc , acc / total))

# Total 1221 correct 547 acc 0.4480

In [ ]:
prompt_original_with_knowledge_before = open('prompt_original_with_knowledge.txt').read()
print(prompt_original_with_knowledge_before)

In [ ]:
import time
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["A", "B", "C"]
for mention_idx, mention in tqdm(enumerate(social_test), total=len(social_test)):
    answer = mention["answer"]
    prompt_q  = prompt_original_with_knowledge_before + "\nQ: " + mention["query"].split('\\n')[0]+ "\n"
    # prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(mention["cands"]):
        if answer == c:
            answer_gold = answer_id[idx]
        prompt_q += '(%c) %s\n' % (answer_id[idx], c)
    prompt_q += mention["query"].split('\\n')[-1].strip()
    prompt_q += "A:"
    # prompt_q += "\nThe answer is "
    # print(prompt_q)
#     break
    while True:
        try:
            response = completion_with_backoff(model="davinci", 
                                                prompt=prompt_q, 
                                                temperature=0, 
                                                max_tokens=5,
                                            )
            break
        except:
            time.sleep(3)
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].strip().split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
#         print("gold:", answer_gold)
#         print("pred:", answer_pred)
#         print("acc:", acc)
#         print("total:", total)
#         print(prompt_q)
#         print(response)
#         break

print('Total %d correct %d acc %.4f' % (total , acc , acc / total))

# Total 1221 correct 547 acc 0.4480

In [ ]:
prompt_original_simple_instruct = open('prompt_original_simple_instruct.txt').read()
print(prompt_original_simple_instruct)

In [ ]:
import time
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["A", "B", "C"]
for mention_idx, mention in tqdm(enumerate(social_test), total=len(social_test)):
    answer_gold = mention["answerKey"].lower()
    # answer = mention["answerKey"]
    prompt_q  = prompt_original_simple_instruct + "\nQ: " + mention["question"]["stem"] + "\n"
    prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(social_test[0]["question"]["choices"]):
        prompt_q += '(%c) %s\n' % (answer_id[idx], c["text"])
    prompt_q += "\nA:"
#     break
    while True:
        try:
            response = completion_with_backoff(model="davinci", 
                                                prompt=prompt_q, 
                                                temperature=0, 
                                                max_tokens=80,
                                            )
            break
        except:
            time.sleep(3)
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].strip().split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
#         print("gold:", answer_gold)
#         print("pred:", answer_pred)
#         print("acc:", acc)
#         print("total:", total)
#         print(prompt_q)
#         print(response)
#         break

print('Total %d correct %d acc %.4f' % (total , acc , acc / total))

# Total 1221 correct 547 acc 0.4480

In [ ]:
prompt_original = open('prompt_original.txt').read()
print(prompt_original)

In [ ]:
import time
davinci_response_list = [] 
acc = 0
total = 0
answer_id = ["A", "B", "C"]
for mention_idx, mention in tqdm(enumerate(social_test), total=len(social_test)):
    answer = mention["answer"]
    prompt_q  = prompt_original + "\nQ: " + mention["query"].split('\\n')[0]+ "\n"
    # prompt_q += "Answer Choices:\n"
    for idx, c in enumerate(mention["cands"]):
        if answer == c:
            answer_gold = answer_id[idx]
        prompt_q += '(%c) %s\n' % (answer_id[idx], c)
    prompt_q += mention["query"].split('\\n')[-1].strip()
    prompt_q += "\nA:"
    # print(prompt_q)
    while True:
        try:
            response = completion_with_backoff(model="davinci", 
                                                prompt=prompt_q, 
                                                temperature=0, 
                                                max_tokens=80,
                                            )
            break
        except:
            time.sleep(3)
    davinci_response_list.append(response)
    answer_pred = response["choices"][0]["text"].strip().split("(")
    answer_pred = answer_pred[1][0] if len(answer_pred) > 1 else "a"
    acc += 1 if answer_pred == answer_gold else 0
    total += 1
    if total%50 ==0:
        print('Total %d correct %d acc %.4f' % (total , acc , acc / total))
print('Total %d correct %d acc %.4f' % (total , acc , acc / total))